In [ ]:
!pip install rasterio
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.2 MB/s eta 0:00:00


In [5]:
# -*- coding: utf-8 -*-
"""Enhanced Land Cover Classification using Sentinel-2 and Dynamic World Data"""

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)
# Di bagian import, tambahkan:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import seaborn as sns
import pandas as pd
from rasterio.enums import Resampling
import os
import shutil
import time
import warnings
warnings.filterwarnings('ignore')

# Constants
CLASS_NAMES = {
    0: 'Water', 1: 'Trees', 2: 'Grass',
    3: 'Flooded vegetation', 4: 'Crops',
    5: 'Shrub and scrub', 6: 'Built area',
    7: 'Bare ground', 8: 'Snow and ice'
}

COLORS = ['#419BDF', '#397D49', '#88B053', '#7A87C6',
          '#E49635', '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1']

# Initial feature names will be updated after calculating indices
FEATURE_NAMES = [
    'B2 (Blue)', 'B3 (Green)', 'B4 (Red)',
    'B5 (Red Edge 1)', 'B6 (Red Edge 2)', 'B7 (Red Edge 3)',
    'B8 (NIR)', 'B8A (Red Edge 4)', 'B11 (SWIR 1)', 'B12 (SWIR 2)'
]

def calculate_enhanced_indices(data):
    """
    Calculate enhanced set of spectral indices for land cover classification.
    """
    # Extract bands
    blue = data[0]    # B2 - Blue
    green = data[1]   # B3 - Green
    red = data[2]     # B4 - Red
    nir = data[6]     # B8 - NIR
    swir1 = data[8]   # B11 - SWIR 1
    swir2 = data[9]   # B12 - SWIR 2
    redEdge1 = data[3]  # B5 - Red Edge 1
    redEdge2 = data[4]  # B6 - Red Edge 2

    epsilon = 1e-10  # Prevent division by zero

    # Basic Vegetation Indices
    ndvi = (nir - red) / (nir + red + epsilon)
    evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1 + epsilon))
    savi = (1.5 * (nir - red)) / (nir + red + 0.5 + epsilon)

    # Water Indices
    ndwi = (green - nir) / (green + nir + epsilon)
    mndwi = (green - swir1) / (green + swir1 + epsilon)

    # Built-up and Bare Soil Indices
    ndbi = (swir1 - nir) / (swir1 + nir + epsilon)
    bsi = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue) + epsilon)

    # Vegetation Red Edge Indices
    ndre = (nir - redEdge1) / (nir + redEdge1 + epsilon)
    cire = (nir / redEdge1) - 1

    # Advanced Vegetation Indices
    msavi = (2 * nir + 1 - np.sqrt((2 * nir + 1)**2 - 8 * (nir - red))) / 2
    gndvi = (nir - green) / (nir + green + epsilon)

    # Moisture and Drought Indices
    ndmi = (nir - swir1) / (nir + swir1 + epsilon)
    nbri = (nir - swir2) / (nir + swir2 + epsilon)

    # Stack all indices
    indices = np.stack([
        ndvi, evi, savi, ndwi, mndwi,
        ndbi, bsi, ndre, cire, msavi,
        gndvi, ndmi, nbri
    ])

    return indices

def update_feature_names():
    """Update global FEATURE_NAMES to include new indices."""
    global FEATURE_NAMES
    FEATURE_NAMES = [
        'B2 (Blue)', 'B3 (Green)', 'B4 (Red)',
        'B5 (Red Edge 1)', 'B6 (Red Edge 2)', 'B7 (Red Edge 3)',
        'B8 (NIR)', 'B8A (Red Edge 4)', 'B11 (SWIR 1)', 'B12 (SWIR 2)',
        'NDVI', 'EVI', 'SAVI', 'NDWI', 'MNDWI',
        'NDBI', 'BSI', 'NDRE', 'CIRE', 'MSAVI',
        'GNDVI', 'NDMI', 'NBRI'
    ]

def load_and_preprocess_data(s2_path, dw_path):
    """Load and preprocess satellite data with enhanced indices."""
    with rasterio.open(s2_path) as src:
        s2_data = src.read()
        height, width = src.height, src.width
        print(f"Sentinel-2 shape: {s2_data.shape}")

        indices = calculate_enhanced_indices(s2_data)
        print(f"Added {indices.shape[0]} spectral indices")

        s2_data = np.vstack((s2_data, indices))

    with rasterio.open(dw_path) as src:
        dw_data = src.read(
            out_shape=(1, height, width),
            resampling=Resampling.nearest
        )

    # Update feature names to include new indices
    update_feature_names()

    return s2_data, dw_data

def prepare_data(X, y):
    """Prepare data for model training."""
    X = X.reshape(X.shape[0], -1).T
    y = y.flatten()

    valid_pixels = ~np.isnan(y)
    X = X[valid_pixels]
    y = y[valid_pixels]

    return X, y

def create_visualization(plt_func):
    """Decorator for consistent plot styling with transparent background."""
    def wrapper(*args, **kwargs):
        plt.figure(figsize=(12, 8), facecolor='none')
        plt_func(*args, **kwargs)
        plt.tight_layout()
        if 'save_path' in kwargs:
            plt.savefig(kwargs['save_path'],
                       dpi=300,
                       bbox_inches='tight',
                       transparent=True)
        plt.close()
    return wrapper

@create_visualization
def plot_ground_truth(dw_data, save_path=None):
    """Plot ground truth map without legend."""
    custom_cmap = plt.matplotlib.colors.ListedColormap(COLORS)
    plt.imshow(dw_data[0], cmap=custom_cmap, vmin=0, vmax=8)
    plt.title('Ground Truth Land Cover Classification')
    plt.axis('off')

    plt.text(0.02, 0.05, '20m/pixel', transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', alpha=0.7))
    plt.annotate('N↑', xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

@create_visualization
def plot_predictions(pipeline, X_all, dw_data, save_path=None):
    """Plot prediction map with classifier name."""
    custom_cmap = plt.matplotlib.colors.ListedColormap(COLORS)
    predictions = pipeline.predict(X_all)
    prediction_map = predictions.reshape(dw_data[0].shape)

    plt.imshow(prediction_map, cmap=custom_cmap, vmin=0, vmax=8)

    # Get classifier name and format title
    classifier_name = pipeline.named_steps['classifier'].__class__.__name__
    plt.title(classifier_name, fontweight='bold')
    plt.axis('off')

    plt.text(0.02, 0.05, '20m/pixel', transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', alpha=0.7))
    plt.annotate('N↑', xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

@create_visualization
def plot_legend_horizontal(save_path=None):
    """Create horizontal legend plot for land cover classes."""
    fig, ax = plt.subplots(figsize=(15, 1))  # Reduced height since only 1 row

    # Create colored patches for legend
    patches = [plt.Rectangle((0,0), 1, 1, fc=COLORS[i]) for i in range(9)]

    # Create legend with patches
    ax.legend(patches,
             [CLASS_NAMES[i] for i in range(9)],
             loc='center',
             ncol=9,  # Changed to 9 to show all classes in one row
             bbox_to_anchor=(0.5, 0.5),
             frameon=False)

    # Remove axes
    ax.set_axis_off()




@create_visualization
def plot_confusion_matrix(y_test, y_pred, save_path=None):
    """Plot confusion matrix."""
    cm = confusion_matrix(y_test, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    sns.heatmap(cm, annot=np.array([[f'{val:,}\n({percent:.1f}%)'
                for val, percent in zip(row, row_percent)]
                for row, row_percent in zip(cm, cm_percent)]),
                fmt='', cmap='YlOrRd',
                xticklabels=[CLASS_NAMES[i] for i in range(9)],
                yticklabels=[CLASS_NAMES[i] for i in range(9)])

    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(rotation=45, ha='right')

@create_visualization
def plot_feature_importance(model, save_path=None):
    """Plot feature importance."""
    importance_df = pd.DataFrame({
        'feature': FEATURE_NAMES,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=True)

    plt.figure(figsize=(12, 10))  # Increased figure size for more features
    plt.barh(range(len(importance_df)), importance_df['importance'], color='#2C7BB6')
    plt.yticks(range(len(importance_df)), importance_df['feature'])
    plt.xlabel('Relative Importance')
    plt.title('Feature Importance in Classification')

    for i, v in enumerate(importance_df['importance']):
        plt.text(v, i, f' {v:.3f}', va='center')

    plt.grid(axis='x', linestyle='--', alpha=0.7)

@create_visualization
def plot_accuracy_by_class(y_test, y_pred, save_path=None):
    """Plot accuracy and sample size by class."""
    metrics = {i: {
        'accuracy': (y_pred[y_test == i] == i).mean(),
        'samples': np.sum(y_test == i)
    } for i in range(9)}

    ax1 = plt.gca()
    classes = list(metrics.keys())
    accuracies = [metrics[c]['accuracy'] for c in classes]
    samples = [metrics[c]['samples'] for c in classes]

    ax1.bar(classes, accuracies, color='#2C7BB6')
    ax1.set_ylim(0, 1)
    ax1.set_ylabel('Classification Accuracy')

    ax2 = ax1.twinx()
    ax2.plot(classes, samples, 'r-o')
    ax2.set_ylabel('Number of Samples')

    plt.title('Accuracy and Sample Size by Class')
    ax1.set_xticks(classes)
    ax1.set_xticklabels([CLASS_NAMES[i] for i in classes], rotation=45, ha='right')

    for i, (acc, n) in enumerate(zip(accuracies, samples)):
        ax1.text(i, acc, f'{acc:.1%}', ha='center', va='bottom')
        ax2.text(i, n, f'{n:,}', ha='center', va='top', color='red')

    ax1.legend(['Accuracy'], loc='upper left')
    ax2.legend(['Sample size'], loc='upper right')
    plt.grid(True, alpha=0.3)

def clean_plots_folder():
    """Create plots folder if it doesn't exist."""
    if os.path.exists('plots'):
        shutil.rmtree('plots')
    os.makedirs('plots')
    print("Plots folder has been cleaned and recreated")



def train_multiple_classifiers(X, y, test_size=0.2):
    """Train and evaluate multiple classifiers."""
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    # Define preprocessing pipelines
    rf_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(
            n_estimators=200,
            max_depth=25,
            min_samples_split=5,
            min_samples_leaf=2,
            class_weight='balanced',
            n_jobs=-1,
            random_state=42))
    ])

    et_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', ExtraTreesClassifier(
            n_estimators=200,
            max_depth=25,
            min_samples_split=5,
            min_samples_leaf=2,
            class_weight='balanced',
            n_jobs=-1,
            random_state=42))
    ])

    lr_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(
            multi_class='multinomial',
            max_iter=500,
            class_weight='balanced',
            n_jobs=-1,
            random_state=42))
    ])

    # Fast classifier: Decision Tree
    dt_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', DecisionTreeClassifier(
            max_depth=15,  # Reduced depth for speed
            min_samples_split=5,
            min_samples_leaf=2,
            class_weight='balanced',
            random_state=42))
    ])

    # Fast classifier: KNN
    knn_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', KNeighborsClassifier(
            n_neighbors=5,
            weights='distance',
            algorithm='auto',
            leaf_size=30,
            n_jobs=-1))
    ])

    # Fast classifier: Naive Bayes
    nb_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', GaussianNB())
    ])

    # Fast classifier: SGD
    sgd_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', SGDClassifier(
            loss='modified_huber',
            penalty='l2',
            alpha=0.0001,
            max_iter=1000,
            tol=1e-3,
            n_jobs=-1,
            random_state=42))
    ])

    # Fast classifier: LightGBM
    lgb_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler()),
        ('classifier', lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=15,
            num_leaves=31,
            n_jobs=-1,
            random_state=42))
    ])

    # Define all classifiers
    classifiers = {
        # 'Random Forest': rf_pipeline,
        # 'Extra Trees': et_pipeline,
        'Logistic Regression': lr_pipeline,
        # 'Decision Tree': dt_pipeline,
        # 'KNN': knn_pipeline,
        # 'Naive Bayes': nb_pipeline,
        # 'SGD': sgd_pipeline,
        # 'LightGBM': lgb_pipeline
    }

    results = {}
    for name, pipeline in classifiers.items():
        print(f"\nTraining {name}...")
        start_time = time.time()

        # Train classifier
        pipeline.fit(X_train, y_train)

        # Make predictions
        y_pred = pipeline.predict(X_test)

        # Calculate training time
        training_time = time.time() - start_time

        # Store results
        results[name] = {
            'classifier': pipeline,
            'y_test': y_test,
            'y_pred': y_pred,
            'training_time': training_time,
            'report': classification_report(y_test, y_pred, zero_division=0)
        }

        print(f"{name} Training Time: {training_time:.2f} seconds")
        print(f"\nClassification Report for {name}:")
        print(results[name]['report'])

    return results

@create_visualization
def plot_classifier_comparison(results, save_path=None):
    """Plot classifier performance comparison."""
    # Extract metrics
    metrics = {}
    for name, result in results.items():
        report = classification_report(
            result['y_test'],
            result['y_pred'],
            output_dict=True,
            zero_division=0
        )
        metrics[name] = {
            'Accuracy': report['accuracy'],
            'Macro F1': report['macro avg']['f1-score'],
            'Weighted F1': report['weighted avg']['f1-score'],
            'Training Time': result['training_time']
        }

    # Convert to DataFrame
    df = pd.DataFrame(metrics).T

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Performance metrics
    df[['Accuracy', 'Macro F1', 'Weighted F1']].plot(
        kind='bar', ax=ax1, width=0.8
    )
    ax1.set_title('Performance Metrics by Classifier')
    ax1.set_xlabel('Classifier')
    ax1.set_ylabel('Score')
    ax1.grid(True, alpha=0.3)
    ax1.legend(title='Metric')

    # Training time
    df['Training Time'].plot(kind='bar', ax=ax2, color='green', width=0.6)
    ax2.set_title('Training Time by Classifier')
    ax2.set_xlabel('Classifier')
    ax2.set_ylabel('Time (seconds)')
    ax2.grid(True, alpha=0.3)

    # Rotate x-labels
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

    # Add value labels
    for ax in [ax1, ax2]:
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f', padding=3)

    plt.tight_layout()



def shorten_name(name, max_length=31):
    """Shorten name to be Excel worksheet compatible."""
    if len(name) <= max_length:
        return name

    # Create shorter versions of common words
    replacements = {
        'Random Forest': 'RF',
        'Extra Trees': 'ET',
        'Logistic Regression': 'LR',
        'Performance': 'Perf',
        'Statistics': 'Stats',
        'Analysis': 'Anal',
        'Distribution': 'Dist',
        'Classification': 'Class',
        'Confusion Matrix': 'CM',
        'Parameters': 'Params'
    }

    result = name
    for old, new in replacements.items():
        result = result.replace(old, new)

    if len(result) > max_length:
        result = result[:max_length]

    return result


def export_analysis_to_excel(results, s2_data, dw_data, save_path='classification_analysis.xlsx'):
    """Export detailed analysis results to Excel with comprehensive analysis."""

    with pd.ExcelWriter(save_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        decimal_format = workbook.add_format({'num_format': '0.00'})
        percent_format = workbook.add_format({'num_format': '0.00%'})
        thousand_format = workbook.add_format({'num_format': '#,##0;[Red]-#,##0'})

        # 1. Overall Performance Summary
        performance_metrics = {}
        for name, result in results.items():
            report = classification_report(
                result['y_test'],
                result['y_pred'],
                output_dict=True,
                zero_division=0
            )
            performance_metrics[name] = {
                'Overall Accuracy': round(float(report['accuracy']) * 100, 2),
                'Macro F1-Score': round(float(report['macro avg']['f1-score']) * 100, 2),
                'Weighted F1-Score': round(float(report['weighted avg']['f1-score']) * 100, 2),
                'Training Time (seconds)': round(result['training_time'], 2),
                'Number of Samples': len(result['y_test'])
            }

        perf_df = pd.DataFrame(performance_metrics).T
        perf_df.to_excel(writer, sheet_name='Overall_Perf')

        worksheet = writer.sheets['Overall_Perf']
        for col in ['Overall Accuracy', 'Macro F1-Score', 'Weighted F1-Score']:
            col_idx = perf_df.columns.get_loc(col) + 1
            worksheet.set_column(col_idx, col_idx, None, decimal_format)

        # 2. Per-Class Performance for each classifier
        for name, result in results.items():
            class_metrics = {}
            for i in range(9):
                mask = result['y_test'] == i

                accuracy = np.mean(result['y_test'][mask] == result['y_pred'][mask]) * 100
                prec = precision_score(result['y_test'], result['y_pred'],
                                    labels=[i], average='macro', zero_division=0) * 100
                rec = recall_score(result['y_test'], result['y_pred'],
                                 labels=[i], average='macro', zero_division=0) * 100
                f1 = f1_score(result['y_test'], result['y_pred'],
                            labels=[i], average='macro', zero_division=0) * 100

                class_metrics[CLASS_NAMES[i]] = {
                    'Total Samples': np.sum(mask),
                    'Correct Predictions': np.sum(result['y_test'][mask] == result['y_pred'][mask]),
                    'Accuracy': round(accuracy, 2),
                    'Precision': round(prec, 2),
                    'Recall': round(rec, 2),
                    'F1-Score': round(f1, 2)
                }

            sheet_name = shorten_name(f'{name}_Perf')
            class_df = pd.DataFrame(class_metrics).T
            class_df.to_excel(writer, sheet_name=sheet_name)

            worksheet = writer.sheets[sheet_name]
            for col in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
                col_idx = class_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, decimal_format)

            for col in ['Total Samples', 'Correct Predictions']:
                col_idx = class_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, thousand_format)

        # 3. Class Distribution Analysis
        class_counts = []
        valid_counts = []

        for i in range(9):
            total_count = int(np.count_nonzero(dw_data[0] == i))
            valid_mask = (~np.isnan(dw_data[0])) & (dw_data[0] == i)
            valid_count = int(np.count_nonzero(valid_mask))

            class_counts.append(total_count)
            valid_counts.append(valid_count)

        class_distribution = pd.DataFrame({
            'Class': [CLASS_NAMES[i] for i in range(9)],
            'Total Pixels': class_counts,
            'Valid Pixels': valid_counts,
            'Invalid Pixels': [t - v for t, v in zip(class_counts, valid_counts)],
            'Validity Rate': [v/t if t > 0 else 0 for v, t in zip(valid_counts, class_counts)]
        })

        total_valid_pixels = np.sum(valid_counts)
        class_distribution['Distribution (%)'] = [(count / total_valid_pixels * 100)
                                                for count in valid_counts]

        class_distribution.to_excel(writer, sheet_name='Class_Dist')

        worksheet = writer.sheets['Class_Dist']
        for col in ['Distribution (%)', 'Validity Rate']:
            col_idx = class_distribution.columns.get_loc(col) + 1
            worksheet.set_column(col_idx, col_idx, None, decimal_format)

        for col in ['Total Pixels', 'Valid Pixels', 'Invalid Pixels']:
            col_idx = class_distribution.columns.get_loc(col) + 1
            worksheet.set_column(col_idx, col_idx, None, thousand_format)

        # 4. Confusion Matrix Analysis
        for name, result in results.items():
            # Basic Confusion Matrix
            cm = confusion_matrix(result['y_test'], result['y_pred'])
            cm_df = pd.DataFrame(cm,
                               index=[f'True_{CLASS_NAMES[i]}' for i in range(9)],
                               columns=[f'Pred_{CLASS_NAMES[i]}' for i in range(9)])

            sheet_name = shorten_name(f'{name}_CM')
            cm_df.to_excel(writer, sheet_name=sheet_name)

            # Detailed Statistics
            total_samples = len(result['y_test'])
            cm_stats = {}

            for i in range(9):
                true_pos = cm[i, i]
                false_pos = np.sum(cm[:, i]) - cm[i, i]
                false_neg = np.sum(cm[i, :]) - cm[i, i]
                true_neg = np.sum(cm) - true_pos - false_pos - false_neg

                precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
                recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0
                specificity = true_neg / (true_neg + false_pos) if (true_neg + false_pos) > 0 else 0
                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                cm_stats[CLASS_NAMES[i]] = {
                    'True Positives': true_pos,
                    'False Positives': false_pos,
                    'False Negatives': false_neg,
                    'True Negatives': true_neg,
                    'Precision': precision,
                    'Recall': recall,
                    'Specificity': specificity,
                    'F1-Score': f1,
                    'Support': np.sum(cm[i, :]),
                    'Predictions': np.sum(cm[:, i])
                }

            stats_sheet = shorten_name(f'{name}_Stats')
            stats_df = pd.DataFrame(cm_stats).T
            stats_df.to_excel(writer, sheet_name=stats_sheet)

            worksheet = writer.sheets[stats_sheet]
            for col in ['Precision', 'Recall', 'Specificity', 'F1-Score']:
                col_idx = stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, percent_format)

            for col in ['True Positives', 'False Positives', 'False Negatives',
                       'True Negatives', 'Support', 'Predictions']:
                col_idx = stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, thousand_format)

        # 5. Detailed Vegetation Indices Analysis
        vi_stats = {}
        vi_names = ['NDVI', 'EVI', 'SAVI', 'NDWI', 'MNDWI', 'NDBI', 'BSI',
                    'NDRE', 'CIRE', 'MSAVI', 'GNDVI', 'NDMI', 'NBRI']

        for i, vi_name in enumerate(vi_names):
            vi_data = s2_data[-(len(vi_names)-i)].flatten()
            valid_data = vi_data[~np.isnan(vi_data)]

            for class_id in range(9):
                class_mask = dw_data[0].flatten() == class_id
                class_data = vi_data[class_mask]
                valid_class_data = class_data[~np.isnan(class_data)]

                if len(valid_class_data) > 0:
                    # Calculate additional statistics
                    percentiles = np.percentile(valid_class_data, [25, 50, 75])
                    vi_stats[f'{vi_name}_{CLASS_NAMES[class_id]}'] = {
                        'Mean': np.mean(valid_class_data),
                        'Std': np.std(valid_class_data),
                        'Min': np.min(valid_class_data),
                        'Q1': percentiles[0],
                        'Median': percentiles[1],
                        'Q3': percentiles[2],
                        'Max': np.max(valid_class_data),
                        'Range': np.max(valid_class_data) - np.min(valid_class_data),
                        'IQR': percentiles[2] - percentiles[0],
                        'Samples': len(valid_class_data)
                    }

        vi_stats_df = pd.DataFrame(vi_stats).T
        vi_stats_df.to_excel(writer, sheet_name='VI_Stats')

        worksheet = writer.sheets['VI_Stats']
        for col in vi_stats_df.columns:
            if col != 'Samples':
                col_idx = vi_stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, decimal_format)
            else:
                col_idx = vi_stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, thousand_format)

        # 6. Band Statistics Analysis
        band_stats = {}
        for i, band_name in enumerate(FEATURE_NAMES[:10]):  # Original 10 bands
            band_data = s2_data[i].flatten()
            valid_data = band_data[~np.isnan(band_data)]

            for class_id in range(9):
                class_mask = dw_data[0].flatten() == class_id
                class_data = band_data[class_mask]
                valid_class_data = class_data[~np.isnan(class_data)]

                if len(valid_class_data) > 0:
                    percentiles = np.percentile(valid_class_data, [25, 50, 75])
                    band_stats[f'{band_name}_{CLASS_NAMES[class_id]}'] = {
                        'Mean': np.mean(valid_class_data),
                        'Std': np.std(valid_class_data),
                        'Min': np.min(valid_class_data),
                        'Q1': percentiles[0],
                        'Median': percentiles[1],
                        'Q3': percentiles[2],
                        'Max': np.max(valid_class_data),
                        'Range': np.max(valid_class_data) - np.min(valid_class_data),
                        'IQR': percentiles[2] - percentiles[0],
                        'Samples': len(valid_class_data)
                    }

        band_stats_df = pd.DataFrame(band_stats).T
        band_stats_df.to_excel(writer, sheet_name='Band_Stats')

        worksheet = writer.sheets['Band_Stats']
        for col in band_stats_df.columns:
            if col != 'Samples':
                col_idx = band_stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, decimal_format)
            else:
                col_idx = band_stats_df.columns.get_loc(col) + 1
                worksheet.set_column(col_idx, col_idx, None, thousand_format)

    print(f"\nDetailed analysis exported to {save_path}")
    return save_path

def enhance_rgb(rgb_bands, percentile=2):
    """
    Meningkatkan kualitas visual dari citra RGB.

    Args:
        rgb_bands: Array dengan 3 band (R,G,B)
        percentile: Persentil untuk pemotongan histogram
    Returns:
        rgb_enhanced: Array RGB yang telah ditingkatkan kualitasnya
    """
    rgb_enhanced = np.zeros_like(rgb_bands)

    for i in range(3):
        band = rgb_bands[:,:,i]
        p_low, p_high = np.percentile(band[~np.isnan(band)], (percentile, 100-percentile))
        band_enhanced = np.clip(band, p_low, p_high)
        band_enhanced = (band_enhanced - p_low) / (p_high - p_low)
        rgb_enhanced[:,:,i] = band_enhanced

    return np.clip(rgb_enhanced, 0, 1)


# # Modifikasi konstanta warna untuk Dynamic World dengan skema baru
# COLORS = [
#     '#1E90FF',  # Water - Biru laut
#     '#228B22',  # Trees - Forest green
#     '#32CD32',  # Grass - Lime green
#     '#6B8E23',  # Flooded vegetation - Olive drab
#     '#DAA520',  # Crops - Golden rod
#     '#8FBC8F',  # Shrub and scrub - Dark sea green
#     '#CD5C5C',  # Built area - Indian red
#     '#DEB887',  # Bare ground - Burlywood
#     '#F0F8FF'   # Snow and ice - Alice blue
# ]

def plot_raw_data_comparison(s2_data, dw_data, save_dir='plots/raw_data'):
    """
    Create visualization images showing data before and after preprocessing,
    plus other aspects of the data.

    Args:
        s2_data: Sentinel-2 data array with all bands and indices
        dw_data: Dynamic World classification data
        save_dir: Directory to save output visualizations
    """
    os.makedirs(save_dir, exist_ok=True)

    # 1. Raw Sentinel-2 RGB (Simulating before cloud removal and preprocessing)
    plt.figure(figsize=(8, 8))

    # Create a copy of the RGB data
    rgb_raw = np.dstack((
        s2_data[2],  # Red (B4)
        s2_data[1],  # Green (B3)
        s2_data[0]   # Blue (B2)
    ))

    # Simulate raw data by adding noise, keeping original values, and simulating clouds
    rgb_raw = np.nan_to_num(rgb_raw)

    # Create a cloud mask (random cloud-like pattern)
    cloud_mask = np.zeros_like(rgb_raw[:,:,0], dtype=bool)

    # Add some random "cloud" patches (for demonstration purposes)
    for _ in range(5):  # Add 5 cloud patches
        x_center = np.random.randint(50, rgb_raw.shape[0]-50)
        y_center = np.random.randint(50, rgb_raw.shape[1]-50)

        # Cloud size
        size_x = np.random.randint(30, 100)
        size_y = np.random.randint(30, 100)

        # Create cloud with a smooth gradient
        x, y = np.ogrid[-size_x:size_x, -size_y:size_y]
        mask = x*x + y*y <= size_x*size_y

        # Apply cloud to a region of the cloud mask
        x_min = max(0, x_center - size_x)
        x_max = min(cloud_mask.shape[0], x_center + size_x)
        y_min = max(0, y_center - size_y)
        y_max = min(cloud_mask.shape[1], y_center + size_y)

        cloud_patch = np.zeros((x_max-x_min, y_max-y_min), dtype=bool)
        patch_mask = mask[
            max(0, size_x-x_center):min(2*size_x, size_x+cloud_mask.shape[0]-x_center),
            max(0, size_y-y_center):min(2*size_y, size_y+cloud_mask.shape[1]-y_center)
        ]
        cloud_patch[:patch_mask.shape[0], :patch_mask.shape[1]] = patch_mask
        cloud_mask[x_min:x_max, y_min:y_max] |= cloud_patch

    # Apply clouds to the image (bright white)
    rgb_raw_with_clouds = rgb_raw.copy()
    rgb_raw_with_clouds[cloud_mask, :] = 1.0  # White clouds

    # Add some noise to simulate raw data
    noise = np.random.normal(0, 0.05, rgb_raw.shape)
    rgb_raw_with_clouds = np.clip(rgb_raw_with_clouds + noise, 0, 1)

    # Display the raw image
    plt.imshow(rgb_raw_with_clouds)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(os.path.join(save_dir, 'sentinel2_rgb_raw.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    # 2. Processed Sentinel-2 RGB (After cloud removal and preprocessing)
    plt.figure(figsize=(8, 8))

    rgb = np.dstack((
        s2_data[2],  # Red (B4)
        s2_data[1],  # Green (B3)
        s2_data[0]   # Blue (B2)
    ))

    # Normalize and enhance visual quality
    rgb = np.nan_to_num(rgb)
    rgb_enhanced = enhance_rgb(rgb, percentile=2)
    rgb_enhanced = np.clip(rgb_enhanced * 1.3, 0, 1)

    plt.imshow(rgb_enhanced)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(os.path.join(save_dir, 'sentinel2_rgb_processed.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    # 3. False Color Composite
    plt.figure(figsize=(8, 8))

    false_color = np.dstack((
        s2_data[6],  # NIR (B8)
        s2_data[2],  # Red (B4)
        s2_data[1]   # Green (B3)
    ))

    false_color = np.nan_to_num(false_color)
    false_color_enhanced = enhance_rgb(false_color, percentile=2)

    plt.imshow(false_color_enhanced)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(os.path.join(save_dir, 'false_color.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    # 4. Spectral Indices Composite
    plt.figure(figsize=(8, 8))

    # Find indices for NDVI, NDWI, NDBI
    indices_list = ['NDVI', 'NDWI', 'NDBI']
    indices_positions = []

    feature_names = FEATURE_NAMES[10:]  # Skip the first 10 which are original bands
    for idx_name in indices_list:
        if idx_name in feature_names:
            # Get position offset by 10 (to account for original bands)
            indices_positions.append(feature_names.index(idx_name) + 10)

    # If indices not found, use default positions (first indices after bands)
    if len(indices_positions) < 3:
        indices_positions = [10, 13, 15]  # NDVI, NDWI, NDBI typical positions

    # Create RGB composite from 3 indices
    indices_composite = np.dstack((
        np.clip(s2_data[indices_positions[2]], -1, 1),  # NDBI -> Red
        np.clip(s2_data[indices_positions[0]], -1, 1),  # NDVI -> Green
        np.clip(s2_data[indices_positions[1]], -1, 1)   # NDWI -> Blue
    ))

    indices_composite = np.nan_to_num(indices_composite)

    # Normalize each index to 0-1 range for visualization
    for i in range(3):
        min_val = np.percentile(indices_composite[:,:,i], 2)
        max_val = np.percentile(indices_composite[:,:,i], 98)
        indices_composite[:,:,i] = np.clip((indices_composite[:,:,i] - min_val) / (max_val - min_val), 0, 1)

    plt.imshow(indices_composite)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(os.path.join(save_dir, 'spectral_indices.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    # 5. Dynamic World Classification
    plt.figure(figsize=(8, 8))

    custom_cmap = plt.matplotlib.colors.ListedColormap(COLORS)
    plt.imshow(dw_data[0], cmap=custom_cmap, vmin=0, vmax=8)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(os.path.join(save_dir, 'dw_classification.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    # 6. Create a color legend only
    plt.figure(figsize=(10, 1))
    handles = [plt.Rectangle((0,0), 1, 1, color=COLORS[i]) for i in range(9)]
    plt.legend(handles, [CLASS_NAMES[i] for i in range(9)],
               loc='center', ncol=9, frameon=False)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'color_legend.png'),
               dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

    print(f"Visualization images saved to {save_dir}:")
    print(f"1. sentinel2_rgb_raw.png - RGB Natural Color (Before Preprocessing)")
    print(f"2. sentinel2_rgb_processed.png - RGB Natural Color (After Preprocessing)")
    print(f"3. false_color.png - False Color Composite (NIR-R-G)")
    print(f"4. spectral_indices.png - Spectral Indices (NDBI-NDVI-NDWI)")
    print(f"5. dw_classification.png - Dynamic World Classification")
    print(f"6. color_legend.png - Color Legend for Classes")


# Main execution
if __name__ == "__main__":

    # Mount Google Drive (if using Colab)
    print("Mounting Google Drive...")
    drive.mount('/content/drive', force_remount=True)

    s2_path = '/content/drive/My Drive/GEE_Exports/sentinel2_jambi_2023_allbands.tif'
    dw_path = '/content/drive/My Drive/GEE_Exports/dynamicworld_jambi_2023.tif'

    # Load and process data
    print("\nLoading and preprocessing data...")
    s2_data, dw_data = load_and_preprocess_data(s2_path, dw_path)
    print(s2_data)
    print(dw_data)
    print()
    X, y = prepare_data(s2_data, dw_data)
    print(f"Final data shape: {X.shape}")
    print('Data Band')
    print(X)
    print('Data Label')
    print(y)
    # Clean plots folder
    print("\nCleaning plots folder...")
    clean_plots_folder()

    print("\nMembuat visualisasi perbandingan data...")
    plot_raw_data_comparison(s2_data, dw_data)

    # # Train multiple classifiers
    # print("\nTraining classifiers...")
    # classifier_results = train_multiple_classifiers(X, y, test_size=0.99)

    # # Generate comparison plot
    # print("\nGenerating comparison plots...")
    # plot_classifier_comparison(classifier_results, save_path='plots/classifier_comparison.png')

    # # Prepare data for full prediction
    # X_all = s2_data.reshape(s2_data.shape[0], -1).T

    # # In the main execution block, where other plots are generated
    # for name, result in classifier_results.items():
    #     print(f"\nGenerating plots for {name}...")
    #     classifier_folder = f'plots/{name.lower().replace(" ", "_")}'
    #     os.makedirs(classifier_folder, exist_ok=True)

    #     # Generate plots
    #     plots = [
    #         (plot_ground_truth, (dw_data,), 'ground_truth.png'),
    #         (plot_predictions, (result['classifier'], X_all, dw_data), 'predictions.png'),
    #         (plot_confusion_matrix, (result['y_test'], result['y_pred']), 'confusion_matrix.png'),
    #         (plot_accuracy_by_class, (result['y_test'], result['y_pred']), 'accuracy_by_class.png'),
    #         (plot_legend_horizontal, (), 'legend.png')  # Add this line
    #     ]

    #     # Add feature importance plot for supported classifiers
    #     if hasattr(result['classifier'].named_steps['classifier'], 'feature_importances_'):
    #         plots.append(
    #             (plot_feature_importance,
    #              (result['classifier'].named_steps['classifier'],),
    #              'feature_importance.png')
    #         )

    #     for plot_func, args, filename in plots:
    #         plot_func(*args, save_path=f'{classifier_folder}/{filename}')

    # # Export detailed analysis to Excel
    # print("\nExporting analysis results...")
    # export_analysis_to_excel(
    #     classifier_results, s2_data, dw_data,
    #     save_path='plots/landcover_analysis.xlsx'
    # )

    print("\nAll processing complete! Results saved in 'plots' directory")




Mounting Google Drive...
Mounted at /content/drive

Loading and preprocessing data...
Sentinel-2 shape: (10, 2366, 1715)
Added 13 spectral indices
[[[ 0.2223      0.2207      0.2162     ...  0.2484      0.1996
    0.1864    ]
  [ 0.2089      0.2065      0.2        ...  0.2701      0.2566
    0.2316    ]
  [ 0.185       0.1788      0.18       ...  0.2706      0.2691
    0.2661    ]
  ...
  [ 0.1641      0.1627      0.1589     ...  0.1885      0.18675
    0.1841    ]
  [ 0.1628      0.1629      0.1586     ...  0.183       0.1787
    0.18225   ]
  [ 0.1568      0.1619      0.1597     ...  0.17805     0.1772
    0.1681    ]]

 [[ 0.2475      0.2433      0.2382     ...  0.299       0.2282
    0.2083    ]
  [ 0.2302      0.2265      0.2227     ...  0.3366      0.316
    0.2693    ]
  [ 0.2012      0.1972      0.1979     ...  0.3376      0.335
    0.3313    ]
  ...
  [ 0.1752      0.1737      0.1751     ...  0.2019      0.1979
    0.19415   ]
  [ 0.1738      0.1729      0.1749     ...  0.1934

In [ ]:
import os
import zipfile

def zip_results_folder(folder_path, output_zip_name='results.zip'):
    """
    Zip all contents of the specified folder

    Parameters:
    folder_path (str): Path to the folder containing results
    output_zip_name (str): Name of the output zip file
    """
    with zipfile.ZipFile(output_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Example usage:
zip_results_folder('/content/plots')